# Benchmarking IQM Spark

This notebook allows you to run some useful benchmarks for the Spark system. Before starting, make sure you have installed all the necessary packages:

In [ ]:
!pip install iqm-benchmarks
!pip install ipykernel

## Connect to the backend

In [ ]:
import os
from iqm.qiskit_iqm import IQMProvider

os.environ["IQM_TOKENS_FILE"]="YOUR TOKEN HERE"
iqm_url =  'YOUR URL HERE'
provider = IQMProvider(iqm_url)
backend = provider.get_backend()

We can access the Spark backend and plot its connectivity graph to check that everything is working properly.

In [ ]:
from rustworkx.visualization import mpl_draw
from rustworkx import spring_layout

mpl_draw(backend.coupling_map.graph.to_undirected(multigraph=False), arrows=True, with_labels=True, node_color='#32a8a4', pos=spring_layout(backend.coupling_map.graph, num_iter=200))

In [4]:
import warnings
warnings.filterwarnings(action="ignore")  

# GHZ state fidelity

The GHZ (Greenberger-Horne-Zeilinger) state is a maximally entangled quantum state that involves three or more qubits. It is an equal superposition of all qubits being in state 0 and all qubits being in state 1.

The GHZ state fidelity acts as a **witness** for genuine multi-qubit entanglement if found to be above $0.5$. This means that the measurement results cannot be explained without entanglement involving **all** qubits, so it is a great way to evaluate the "quantumness" of the computer. 

The fidelity of a mixed state is computed using the [formula](https://en.wikipedia.org/wiki/Fidelity_of_quantum_states) 
$$F(\text{ideal}, \text{measured})= \left(\text{Tr}\sqrt{\sqrt{\rho_{\text{ideal}}}\rho_{\text{measured}}\sqrt{\rho_{\text{ideal}}}}\right)^2$$

where $\rho_{\text{ideal}}$ is the density matrix of an ideal GHZ state (i.e. without noise) and $\rho_{\text{measured}}$ is the density matrix as given by the actual results of the quantum computer. The ideal GHZ state density matrix only has non-zero entries in the four corners of the density matrix $|00...0\rangle\langle00...0|$, $|00...0\rangle \langle11...1|$, $|11...1\rangle \langle00...0|$, $|11...1\rangle \langle11...1|$. This simplifies the process since we only need to measure these four components. In the fidelity formula, all other entries are effectively nullified by the zero entries in the ideal state matrix. To measure the coherences (off-diagonal entries) we use the method of multiple quantum coherences [Mooney, 2021](https://iopscience.iop.org/article/10.1088/2399-6528/ac1df7/meta). 

In [5]:
from iqm.benchmarks.entanglement.ghz import GHZConfiguration, GHZBenchmark

In [6]:
GHZ = GHZConfiguration(
    state_generation_routine="tree",
    custom_qubits_array=[[0,1,2,3,4]],
    shots=1000,
    qiskit_optim_level=3,
    optimize_sqg=True,
    fidelity_routine="coherences", 
    rem=True,
    mit_shots=1000,
)

> If you want to modify the settings above, please refer to the documentation [here](https://iqm-finland.github.io/iqm-benchmarks/api/iqm.benchmarks.entanglement.ghz.GHZConfiguration.html#iqm.benchmarks.entanglement.ghz.GHZConfiguration).

Before running the benchmark analysis, we can visualize the histogram of counts obtained from measuring a GHZ state on 5 qubits:

In [ ]:
from iqm.benchmarks.entanglement.ghz import generate_ghz_spanning_tree, get_edges
from qiskit import transpile
from qiskit.visualization import plot_histogram

qubit_layout = [0,1,2,3,4]
graph = get_edges(coupling_map=backend.coupling_map, qubit_layout=qubit_layout)
ghz_circuit = generate_ghz_spanning_tree(graph, qubit_layout, n_state=5)[0]
qc_transp = transpile(ghz_circuit, backend=backend, optimization_level=3)
res = backend.run(qc_transp, shots=10000).result() 
counts=res.get_counts()

plot_histogram(counts)

In [ ]:
benchmark_ghz = GHZBenchmark(backend, GHZ)
run_ghz = benchmark_ghz.run()

In [ ]:
result_ghz = benchmark_ghz.analyze()
for observation in result_ghz.observations:
    if observation.identifier.string_identifier == str(qubit_layout):
        print(f"{observation.name}: {observation.value}") 

# Quantum Volume

Quantum volume is a single-number metric that was introduced in [Cross, 2019](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.100.032328). It evaluates the quality of a quantum processor via the largest random **square** circuit that it can run successfully. 

The success of a run is based on the heavy output probability, which corresponds to the probability of observing *heavy outputs*, i.e. the measurement outputs that occcur with a probability greater than the median of the distribution. The heavy output generation problem asks if the generated distribution of the random circuit we run contains heavy outputs at least 2/3 of the time (on average) with a high confidence level, typically higher than 97.5%. It can be shown that the heavy output probability for an ideal device is at around 0.85 asymptotically. 
The quantum volume is then defined as
$$ \log_2 V_q = \underset{n}{\argmax} \min (n, d(n))$$
where $n \leq N$ is a number of qubits and $d(n)$ is the *achievable depth*, i.e. the largest depth such that we are confident the probability of observing a heavy output is greater than 2/3.


In [10]:
from iqm.benchmarks.quantum_volume.quantum_volume import QuantumVolumeConfiguration, QuantumVolumeBenchmark

We define a combination of qubits to test quantum volume on. Due to the star topology, the combinations must contain at least qubit #2 (see topmost graph).

In [11]:
QV = QuantumVolumeConfiguration(
    num_circuits=500, 
    shots=2**8,
    calset_id=None,
    num_sigmas=2,
    choose_qubits_routine="custom",
    custom_qubits_array=[[0,1,2], [0,2,3], [0,2,4], [1,2,3], [1,2,4]], 
    # custom_qubits_array=[[0,1,2,3], [0,1,2,4], [0,2,3,4], [1,2,3,4]],
    qiskit_optim_level=3,
    optimize_sqg=True,
    max_gates_per_batch=60_000,
    rem=True,
    mit_shots=1_000,
)

> If you want to modify the settings above, please refer to the documentation [here](https://iqm-finland.github.io/iqm-benchmarks/api/iqm.benchmarks.quantum_volume.quantum_volume.QuantumVolumeConfiguration.html#iqm.benchmarks.quantum_volume.quantum_volume.QuantumVolumeConfiguration).

Warning: The following code cell may take few minutes to run.

In [ ]:
benchmark_qv = QuantumVolumeBenchmark(backend, QV)
run_qv = benchmark_qv.run()

In [ ]:
result_qv = benchmark_qv.analyze()
for v in result_qv.plots.values():
    display(v)

# Circuit Layer Operations Per Second (CLOPS)

CLOPS is a metric that quantifies the speed at which a quantum computer can execute the layers of a quantum circuit ([Wack, 2021](https://arxiv.org/abs/2110.14108)). It is measured by counting the number of complete layers of gates that can be implemented per second. 

The circuits run to calculate this benchmark have the same structure as the ones used for the Quantum Volume (QV).

More precisely, CLOPS results from the ratio between the total number of QV layers executed and the **total** execution time, which includes also the submission of the circuits to the QPU and the processing of the results.

In [14]:
from iqm.benchmarks.quantum_volume.clops import CLOPSConfiguration, CLOPSBenchmark, plot_times

In [15]:
CLOPS = CLOPSConfiguration(
    qubits=[0,1,2],
    num_circuits=100,
    num_updates=10, 
    num_shots=100, 
    calset_id=None,
    qiskit_optim_level=3,
    optimize_sqg=True,
    routing_method="sabre",
    physical_layout="fixed",
)

> If you want to modify the settings above, please refer to the documentation [here](https://iqm-finland.github.io/iqm-benchmarks/api/iqm.benchmarks.quantum_volume.clops.CLOPSConfiguration.html#iqm.benchmarks.quantum_volume.clops.CLOPSConfiguration).

In [ ]:
benchmark_clops = CLOPSBenchmark(backend, CLOPS)
run_clops = benchmark_clops.run()

In [ ]:
result_clops = benchmark_clops.analyze()
fig_clops = plot_times(result_clops.dataset, result_clops.observations)
display(fig_clops[1])

# Clifford Randomized Benchmarking

The idea behind Clifford Randomized Benchmarking (CRB) is that under certain (simplified) types of noise, the average survival probability of the initial state of a quantum system under uniformly random sequences of multi-qubit Clifford gates with sequence inversion will decay exponentially in the length of the sequences. From such decay, one can in turn infer the average fidelity of the corresponding Clifford group.

The main assumption we will make here is that the noise can be modeled as Markovian, time-stationary and gate-independent. 

The theory of RB and the fact that the multi-qubit Clifford group is a **unitary 2-design**, ensures that the average fidelity of our gate set is given by
$$\overline{F}_\text{CRB}=p+2^{-n}(1-p)$$

CRB is not generally intended to work for $n>2$, both because of the scaling of the size of the $n$-qubit Clifford group in $n$, and because such gates have to eventually be transpiled to a native basis of 1Q and 2Q gates!

It is important to underline that the clifford fidelity is related to the average fidelity of the native gate set as: 
$$\overline{F}_\text{GATE} \approx 1 - \frac{1-\overline{F}_\text{CRB}}{1.875}$$
This means that all the single-qubit Clifford gates can be decomposed using - on average - 1.875 gates from the native set. This formula shows that the value of $\overline{F}_\text{GATE}$ will always be slightly higher than $\overline{F}_\text{CRB}$, resulting in a difference between the fidelity here calculated and those reported usually in the specs of a QPU. 

In [18]:
from iqm.benchmarks.randomized_benchmarking.clifford_rb.clifford_rb import CliffordRBConfiguration,CliffordRandomizedBenchmarking

In [19]:
CRB = CliffordRBConfiguration(
    qubits_array=[[0],[1],[2],[3],[4]],
    sequence_lengths=[2**(m+1)-1 for m in range(9)],
    num_circuit_samples=25,
    shots=2**8,
    calset_id=None,
    parallel_execution=False,
)

> If you want to modify the settings above, please refer to the documentation [here](https://iqm-finland.github.io/iqm-benchmarks/api/iqm.benchmarks.randomized_benchmarking.clifford_rb.clifford_rb.CliffordRBConfiguration.html#iqm.benchmarks.randomized_benchmarking.clifford_rb.clifford_rb.CliffordRBConfiguration).

Warning: The following code cell may take few minutes to run.

In [ ]:
benchmark_clifford_rb = CliffordRandomizedBenchmarking(backend, CRB)
run_clifford_rb = benchmark_clifford_rb.run()

In [ ]:
result_clifford_rb = benchmark_clifford_rb.analyze()
for plot in result_clifford_rb.plots.values():
    display(plot)

# Interleaved Randomized Benchmarking (IRB)

Differently from the previous protocol, this benchmark aims at estimating the average fidelity of an **individual** quantum gate. This can be achieved interleaving random Clifford gates between the gate of interest. 

The protocol then runs the two sequences (random Cliffords with and without interleaved gate) and extracts the decay parameters (we expect the one for IRB to be smaller than the CRB one because the sequence is longer). The average fidelity of the gate we wish to characterize is then calculated using the two decay parameters. 

The method was introduced in [Magesan, 2012](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.109.080505) and it is robust with respect to state preparation and measurement errors.

In [22]:
from iqm.benchmarks.randomized_benchmarking.interleaved_rb.interleaved_rb import InterleavedRBConfiguration, InterleavedRandomizedBenchmarking

In [23]:
IRB_CZ = InterleavedRBConfiguration(
    qubits_array=[[0,2],[1,2],[2,3],[2,4]],
    sequence_lengths=[2**(m+1)-1 for m in range(7)],
    num_circuit_samples=25,
    shots=2**8,
    calset_id=None,
    parallel_execution=False,
    interleaved_gate = "CZGate",
    interleaved_gate_params = None,
    simultaneous_fit = ["amplitude", "offset"],
)

> If you want to modify the settings above, please refer to the documentation [here](https://iqm-finland.github.io/iqm-benchmarks/api/iqm.benchmarks.randomized_benchmarking.interleaved_rb.interleaved_rb.InterleavedRBConfiguration.html#iqm.benchmarks.randomized_benchmarking.interleaved_rb.interleaved_rb.InterleavedRBConfiguration).

NB: Clifford RB is executed by default when running Interleaved RB!

Warning: The following code cells may take **several** minutes to run.

In [ ]:
benchmark_irb_CZ = InterleavedRandomizedBenchmarking(backend, IRB_CZ)
run_irb_CZ = benchmark_irb_CZ.run()

In [ ]:
result_irb_CZ = benchmark_irb_CZ.analyze()
for plot in result_irb_CZ.plots.values():
    display(plot)

# Q-Score

*The Q-score measures the maximum number of qubits that can be used
effectively to solve the MaxCut combinatorial optimization problem with the Quantum Approximate
Optimization Algorithm* - [Martiel,2021](https://ieeexplore.ieee.org/document/9459509)

The graphs chosen for the benchmark are random Erdős-Rényi graphs with 50% edge-probability between nodes.
The obtained cost of the solution, i.e. the average number of cut edges, must be above a certain threshold. Specifically, one has to find the cost of a graph to be above $\beta\geq 0.2$ on a scale where $\beta = 0$ corresponds to a random solution and $\beta = 1$ to an ideal solution. 

In [ ]:
from iqm.benchmarks.optimization.qscore import QScoreConfiguration, QScoreBenchmark

In [ ]:
QSCORE = QScoreConfiguration(
    num_instances = 100,
    num_qaoa_layers= 1,
    shots = 2**8,
    calset_id=None, 
    min_num_nodes = 2,
    max_num_nodes = 5,
    use_virtual_node = True,
    use_classically_optimized_angles = True,
    choose_qubits_routine = "custom",
    custom_qubits_array=[[2],
                    [2, 0],
                    [2, 0, 1],
                    [2, 0, 1, 3],
                    [2, 0, 1, 3, 4]],
    seed = 1
    )

> If you want to modify the settings above, please refer to the documentation [here](https://iqm-finland.github.io/iqm-benchmarks/api/iqm.benchmarks.optimization.qscore.QScoreConfiguration.html#iqm.benchmarks.optimization.qscore.QScoreConfiguration).

Warning: The following code cell may take **several** minutes to run.

In [ ]:
benchmark_qscore = QScoreBenchmark(backend, QSCORE)
run_qscore = benchmark_qscore.run()

In [ ]:
result_qscore = benchmark_qscore.analyze()

In [ ]:
result_qscore.plot_all()

# Summary

Typical performance for IQM Spark is summarized in the table below and compared to the values obtained with your device. The typical fidelities reported below refer to the median over the 5 qubits of the system.

In [ ]:
import numpy as np

### GHZ
obs_ghz = result_ghz.observations
fidelity = round(min([obs_ghz[i].value for i in range(len(obs_ghz)) if obs_ghz[i].name=='fidelity']),2)

### QV
obs_qv = result_qv.observations
qv = max([obs_qv[i].value for i in range(len(obs_qv)) if obs_qv[i].name=='QV_result'])

### CLOPS
obs_clops = result_clops.observations
clops = max([obs_clops[item]['clops_v']['value'] for item in obs_clops])

### CRB
obs_crb = result_clifford_rb.observations
f_crb = round(np.median([obs_crb[i].value for i in range(len(obs_crb))]),3)

### IRB
obs_irb = result_irb_CZ.observations
f_irb = round(np.median([obs_irb[i].value for i in range(len(obs_irb)) if obs_irb[i].name=='avg_gate_fidelity_interleaved']),3)

### QS 
obs_qs = result_qscore.observations
qs = np.argmin([obs_qs[i].value-0.2 for i in range(len(obs_qs)) if obs_qs[i].name == 'approximation_ratio' and obs_qs[i].value-0.2>0])+2


summary = {'5-qubit GHZ state fidelity': ['≥ 0.5', fidelity],
    'Quantum Volume': ['≥ 8', qv], 
    'CLOPS': ['3000', clops], 
    'Single-qubit gate fidelity': ['≥ 0.999', f_crb],
    'Two-qubit gate (CZ) fidelity': ['≥ 0.98', f_irb], 
    'Q-Score': ['≥ 5', qs] 
}

print("{:<30} {:<15} {:<15}".format('Benchmark', 'Typical', 'Your device'))
for k, v in summary.items():
    label, num = v
    print("{:<30} {:<15} {:<15}".format(k, label, num))